In [1]:
# define the model
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.applications.inception_v3 import preprocess_input
from keras.models import Model, Sequential
from keras.layers import Input, Dense, concatenate, Activation, Add, Lambda, Conv2D, MaxPooling2D, Flatten
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Dropout
from keras import backend as K
from keras import optimizers
from keras.utils import to_categorical

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 1060 6GB (CNMeM is disabled, cuDNN 5110)


In [6]:
img_a = Input(shape=(3, 256, 256))
img_b = Input(shape=(3, 256, 256))

score_model = Sequential()

score_model.add(Conv2D(filters=32, kernel_size=(11, 11), activation="relu", input_shape=(3, 256, 256)))
score_model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
#model.add(BatchNormalization())
score_model.add(Conv2D(filters=32, kernel_size=(5, 5), activation="relu"))
score_model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
#model.add(BatchNormalization())
score_model.add(Conv2D(filters=64, kernel_size=(3, 3), activation="relu"))
score_model.add(Conv2D(filters=64, kernel_size=(3, 3), activation="relu"))
score_model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
#score_model.add(Conv2D(filters=64, kernel_size=(3, 3), activation="relu"))
#score_model.add(Conv2D(filters=64, kernel_size=(3, 3), activation="relu"))
score_model.add(Flatten())
score_model.add(Dense(128, activation='relu'))
#score_model.add(BatchNormalization())
#score_model.add(Dropout(0.5))
score_model.add(Dense(1))

score_a = score_model(img_a)
score_b = score_model(img_b)

negated_score_b = Lambda(lambda x: -x, output_shape=(1,))(score_b)
diff = Add()([score_a, negated_score_b])

output = Activation("sigmoid")(diff)

ranknet = Model(inputs=[img_a, img_b], outputs=output)

optimizer = optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
ranknet.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])
ranknet.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_4 (InputLayer)             (None, 3, 256, 256)   0                                            
____________________________________________________________________________________________________
sequential_2 (Sequential)        (None, 1)             6064929     input_3[0][0]                    
                                                                   input_4[0][0]                    
____________________________________________________________________________________________________
input_3 (InputLayer)             (None, 3, 256, 256)   0                                            
____________________________________________________________________________________________________
lambda_2 (Lambda)                (None, 1)             0           sequential_2[2][0]      

In [3]:
img_path = "../data/img/"
def img_preprocess(imgid, target_size=(299, 299)):
    try:
        filename = img_path+"%s.jpg"%imgid
        img = image.load_img(filename, target_size=target_size)
        x = image.img_to_array(img)
        return x
    except Exception, e:
        print str(e)
        return None

In [ ]:
# RUN ONLY ONCE
import cPickle as pickle
import random
import os
import shutil

train_list = pickle.load(open("./data/ranknet/train.list", 'rb'))
valid_list = pickle.load(open("./data/ranknet/valid.list", 'rb'))

if os.path.exists("./data/ranknet_from_scratch"):
    shutil.rmtree("./data/ranknet_from_scratch")
os.makedirs("./data/ranknet_from_scratch/train/none")
os.makedirs("./data/ranknet_from_scratch/valid/none")

white_set = set()
def touch_files(dataset, path, count=-1):
    if count != -1:
        subdata = random.sample(dataset, count)
    else:
        subdata = dataset
    for imgA, sA, imgB, sB, cmpret in subdata:
        if imgA in white_set or img_preprocess(imgA) is not None:
            white_set.add(imgA)
        if imgB in white_set or img_preprocess(imgB) is not None:
            white_set.add(imgB)
        if imgA in white_set and imgB in white_set:
            open(path+"/%s_%s_%d.jpg"%(imgA, imgB, cmpret), 'w').close()
        
touch_files(train_list, path="./data/ranknet_from_scratch/train/none/", count=100000)
touch_files(valid_list, path="./data/ranknet_from_scratch/valid/none/", count=-1)

In [4]:
from keras.preprocessing.image import *
import os

# 自定义DirectoryIterator类，可以返回自定义的label
class CustomDirectoryIterator(DirectoryIterator):  
    def next(self):
        """For python 2.x.
        # Returns
            The next batch.
        """
        with self.lock:
            index_array, current_index, current_batch_size = next(self.index_generator)
        # The transformation of images is not under thread lock
        # so it can be done in parallel
        batch_x1 = np.zeros((current_batch_size,) + self.image_shape, dtype=K.floatx())
        batch_x2 = np.zeros((current_batch_size,) + self.image_shape, dtype=K.floatx())
        batch_y = np.zeros((current_batch_size, ) , dtype=K.floatx())
        # build batch of image data
        for i, j in enumerate(index_array):
            fname =  self.filenames[j]
            fname = os.path.basename(fname)
            fname, _ = os.path.splitext(fname)
            imgA, imgB, cmpret = fname.split("_")
            x1 = img_preprocess(imgA, self.target_size)
            x2 = img_preprocess(imgB, self.target_size)
            batch_x1[i] = x1
            batch_x2[i] = x2
            batch_y[i] = int(cmpret)
        return [batch_x1, batch_x2], batch_y

# 定义批处理的数据集大小：较小的batch_size可以增加权重调整的次数，同时节省内存的开销
batch_size = 16 

# 图片预处理工具类
IDG = ImageDataGenerator()

# 从目录文件中流式读取数据，避免训练中一次性加载爆内存
train_batch = CustomDirectoryIterator("./data/ranknet_from_scratch/train/", IDG, 
                                      target_size=(256, 256), batch_size=batch_size, shuffle=True)
valid_batch = CustomDirectoryIterator("./data/ranknet_from_scratch/valid/", IDG, 
                                      target_size=(256, 256), batch_size=batch_size, shuffle=True)

Found 99530 images belonging to 1 classes.
Found 4334 images belonging to 1 classes.


In [7]:
ranknet.fit_generator(train_batch, steps_per_epoch=train_batch.samples // batch_size, epochs=10,
                       validation_data=valid_batch, validation_steps=valid_batch.samples // batch_size)

Epoch 1/10
6220/6220 [==============================] - 1636s - loss: 0.7060 - acc: 0.5691 - val_loss: 0.7294 - val_acc: 0.5206
Epoch 2/10
6220/6220 [==============================] - 1626s - loss: 0.4988 - acc: 0.7537 - val_loss: 0.9570 - val_acc: 0.5345
Epoch 3/10
6220/6220 [==============================] - 1625s - loss: 0.2627 - acc: 0.8945 - val_loss: 1.1643 - val_acc: 0.5310
Epoch 4/10
6220/6220 [==============================] - 1626s - loss: 0.1590 - acc: 0.9425 - val_loss: 1.2906 - val_acc: 0.5285
Epoch 5/10
6220/6220 [==============================] - 1632s - loss: 0.1146 - acc: 0.9605 - val_loss: 1.3951 - val_acc: 0.5312
Epoch 6/10
6220/6220 [==============================] - 1671s - loss: 0.0833 - acc: 0.9725 - val_loss: 1.4036 - val_acc: 0.5271
Epoch 7/10
6220/6220 [==============================] - 1716s - loss: 0.0670 - acc: 0.9787 - val_loss: 1.7835 - val_acc: 0.5345
Epoch 8/10
6220/6220 [==============================] - 1646s - loss: 0.0548 - acc: 0.9826 - val_loss: 1

In [ ]:
ranknet.save_weights("./data/ranknet_from_scratch/ranknet_from_scratch.h5")